In [1]:
from modeling_xcodec2 import XCodec2Model
from xcodec2.modeling_xcodec2 import XCodec2Model as XCodec2Model_ori
import torch
import torch.nn.functional as F

torch.autograd.set_grad_enabled(False)

/home/husein/.local/lib/python3.10/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:445: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/husein/.local/lib/python3.10/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:630: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/husein/.local/lib/python3.10/site-packages/vector_quantize_pytorch/finite_scalar_quantization.py:147: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/home/husein/.local/lib/python3.10/site-packages/vector_quantize_pytorch/lookup_free_quantization.py:209: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instea

In [2]:
model_path = "HKUSTAudio/xcodec2"  
 
model = XCodec2Model.from_pretrained(model_path, ignore_mismatched_sizes=True)
model_ori = XCodec2Model_ori.from_pretrained(model_path)

You are using a model of type xcodec2 to instantiate a model of type xcodec. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at HKUSTAudio/xcodec2 were not used when initializing XCodec2Model: ['CodecEnc.conv_blocks.0.weight_g', 'CodecEnc.conv_blocks.0.weight_v', 'CodecEnc.conv_blocks.1.block.0.block.1.weight_g', 'CodecEnc.conv_blocks.1.block.0.block.1.weight_v', 'CodecEnc.conv_blocks.1.block.0.block.3.weight_g', 'CodecEnc.conv_blocks.1.block.0.block.3.weight_v', 'CodecEnc.conv_blocks.1.block.1.block.1.weight_g', 'CodecEnc.conv_blocks.1.block.1.block.1.weight_v', 'CodecEnc.conv_blocks.1.block.1.block.3.weight_g', 'CodecEnc.conv_blocks.1.block.1.block.3.weight_v', 'CodecEnc.conv_blocks.1.block.2.block.1.weight_g', 'CodecEnc.conv_blocks.1.block.2.block.1.weight_v', 'CodecEnc.conv_blocks.1.block.2.block.3.weight_g', 'CodecEnc.conv_blocks.1.block.2.block.3.weight_v', 'CodecEnc.conv_blocks.1.block.4.weight_g', 'CodecEnc.conv_

Some weights of XCodec2Model were not initialized from the model checkpoint at HKUSTAudio/xcodec2 and are newly initialized because the shapes did not match:
- generator.head.istft.window: found shape torch.Size([1280]) in the checkpoint and torch.Size([1920]) in the model instantiated
- generator.head.out.bias: found shape torch.Size([1282]) in the checkpoint and torch.Size([1922]) in the model instantiated
- generator.head.out.weight: found shape torch.Size([1282, 1024]) in the checkpoint and torch.Size([1922, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are using a model of type xcodec2 to instantiate a model of type xcodec. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at HKUSTAudio/xcodec2 were not used when initializing XCodec2Model: ['CodecEnc.conv_blocks.0.weight_g', 'CodecEnc.conv_blocks.0.weight_v', 'CodecEn

In [3]:
s = """
Some weights of XCodec2Model were not initialized from the model checkpoint at HKUSTAudio/xcodec2 and are newly initialized because the shapes did not match:
- generator.head.istft.window: found shape torch.Size([1280]) in the checkpoint and torch.Size([1920]) in the model instantiated
- generator.head.out.bias: found shape torch.Size([1282]) in the checkpoint and torch.Size([1922]) in the model instantiated
- generator.head.out.weight: found shape torch.Size([1282, 1024]) in the checkpoint and torch.Size([1922, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are using a model of type xcodec2 to instantiate a model of type xcodec. This is not supported for all configurations of models and can yield errors.
"""

In [4]:
weight = model_ori.generator.head.out.weight
bias = model_ori.generator.head.out.bias
new_size = model.generator.head.out.weight.shape[0]

In [5]:
weight_1d = weight.T.unsqueeze(0)
weight_resized = F.interpolate(
    weight_1d, size=new_size, mode="linear", align_corners=True
)
weight_resized = weight_resized.squeeze(0).T

bias_1d = bias.unsqueeze(0).unsqueeze(0)
bias_resized = F.interpolate(
    bias_1d, size=new_size, mode="linear", align_corners=True
)
bias_resized = bias_resized.squeeze(0).squeeze(0)

In [6]:
model.generator.head.out.weight

Parameter containing:
tensor([[-0.0085,  0.0098, -0.0070,  ..., -0.0131,  0.0095,  0.0043],
        [-0.0173, -0.0166,  0.0061,  ..., -0.0106, -0.0033, -0.0274],
        [ 0.0223,  0.0069, -0.0306,  ..., -0.0061, -0.0186,  0.0125],
        ...,
        [ 0.0254,  0.0098, -0.0292,  ..., -0.0243,  0.0231,  0.0017],
        [ 0.0080,  0.0301,  0.0090,  ...,  0.0228, -0.0275, -0.0060],
        [-0.0220,  0.0051, -0.0076,  ..., -0.0211, -0.0239, -0.0106]],
       requires_grad=True)

In [7]:
with torch.no_grad():
    model.generator.head.out.weight.copy_(weight_resized)
    model.generator.head.out.bias.copy_(bias_resized)

In [8]:
model.generator.head.out.weight

Parameter containing:
tensor([[ 0.0094,  0.0249,  0.0296,  ..., -0.0329,  0.0683, -0.0308],
        [ 0.0191,  0.0315,  0.0257,  ...,  0.0276,  0.0226,  0.0008],
        [ 0.0133,  0.0064,  0.0632,  ...,  0.0353, -0.0330,  0.0178],
        ...,
        [-0.0146,  0.0378,  0.0402,  ...,  0.0417, -0.0029, -0.0381],
        [-0.0075,  0.0425, -0.0268,  ...,  0.0296,  0.0309, -0.0732],
        [-0.1070,  0.0141, -0.0949,  ...,  0.1110,  0.1289, -0.1221]],
       requires_grad=True)

In [9]:
model.save_pretrained('./interpolate')

[2025-09-18 18:17:52,537] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
collect2: error: ld returned 1 exit status
